In [15]:
import os
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, ForeignKey, event

# === 1. Supprimer ancienne base ===
db_path = "nutriprofil_final.db"
if os.path.exists(db_path):
    os.remove(db_path)

# === 2. Créer l'engine + activer les foreign keys ===
engine = create_engine(f"sqlite:///{db_path}")

@event.listens_for(engine, "connect")
def enforce_foreign_keys(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys = ON")
    cursor.close()

# === 3. Définir les tables ===
metadata = MetaData()

dim_time = Table(
    'dim_time', metadata,
    Column('year_id', Integer, primary_key=True)
)

dim_family = Table(
    'dim_family', metadata,
    Column('family_id', Integer, primary_key=True, autoincrement=True),
    Column('family_name', String, unique=True, nullable=False)
)

dim_food = Table(
    'dim_food', metadata,
    Column('food_id', Integer, primary_key=True, autoincrement=True),
    Column('name', String, nullable=False),
    Column('family_id', Integer, ForeignKey('dim_family.family_id')),
    Column('Fibres', Float),
    Column('Gras', Float),
    Column('Glucides', Float),
    Column('IG', Float),
    Column('CG', Float),
    Column('Proteines', Float)
)

fact_consumption = Table(
    'fact_consumption', metadata,
    Column('year_id', Integer, ForeignKey('dim_time.year_id'), primary_key=True),
    Column('family_id', Integer, ForeignKey('dim_family.family_id'), primary_key=True),
    Column('value', Float, nullable=False)
)

fact_health = Table(
    'fact_health', metadata,
    Column('year_id', Integer, ForeignKey('dim_time.year_id'), primary_key=True),
    Column('age_group_name', String, primary_key=True),
    Column('sex_name', String, primary_key=True),
    Column('diabete_prev', Float),
    Column('obesite_prev', Float),
    Column('hypertension_prev', Float),
    Column('mcv_mort_rate', Float),
    Column('cancercolorectal_mort', Float),
    Column('alzheimer_mort', Float),
    Column('ssbs', Float),
    Column('fiber', Float),
    Column('sodium', Float),
    Column('transfat', Float)
)

food_health_link = Table(
    'food_health_link', metadata,
    Column('link_id', Integer, primary_key=True, autoincrement=True),
    Column('food_id', Integer, ForeignKey('dim_food.food_id')),
    Column('indicateur', String),
    Column('effet', String),
    Column('seuil_description', String),
    Column('direction', String),
    Column('source', String)
)

# === 4. Créer les tables ===
metadata.create_all(engine)

# === 5. Charger les CSV ===
df_cons = pd.read_csv("/Users/coulibalykani/Desktop/Projet 3/nutriprofil/Dataframe/previsions_conso_ML_Lasso.csv")
df_food = pd.read_csv("/Users/coulibalykani/Desktop/Projet 3/nutriprofil/Dataframe/DF_Aliment_NUTRIPROFIL.csv")
df_health = pd.read_csv("/Users/coulibalykani/Desktop/Projet 3/nutriprofil/Dataframe/france_nutrition_sante_fusionfinal.csv")

# === 6. Nettoyage familles (harmonisation) ===
if "Famille de produits" not in df_cons.columns:
    raise KeyError("La colonne 'Famille de produits' est absente du fichier de consommation.")

mapping_familles = {
    'Céréales et produits à base de céréales (riz, pâtes, etc)': 'Céréales',
    'Viandes': 'Produits animaux',
    'Poissons et autres fruits de mer ': 'Produits animaux',
    'Lait, produits laitiers divers et œufs ': 'Produits laitiers',
    'Huiles et graisses ': 'Aides culinaires',
    'Fruits et fruits à coque ': 'Fruits et légumes',
    'Légumes, tubercules, bananes plantains, bananes de cuisine et légumes secs ': 'Fruits et légumes',
    'Sucre, confiserie et desserts ': 'Produits sucrés',
    'Plats préparés et autres produits alimentaires ': 'Plats préparés',
    'Jus de fruits et de légumes ': 'Boissons',
    'Café et succédanés de café ': 'Boissons',
    'Thé, maté et autres produits d’origine végétale pour tisanes ': 'Boissons',
    'Eau ': 'Boissons',
    'Sodas': 'Boissons',
    'Autres boissons non alcoolisées ': 'Boissons',
    'Spiritueux et liqueurs ': 'Boissons',
    'Vin': 'Boissons',
    'Bière ': 'Boissons',
    'Autres boissons alcoolisées ': 'Boissons'
}

df_cons["famille_nette"] = df_cons["Famille de produits"].map(mapping_familles)
df_fam = pd.DataFrame(df_cons["famille_nette"].dropna().unique(), columns=["family_name"])
df_fam = df_fam.drop_duplicates().reset_index(drop=True)
df_fam.to_sql("dim_family", engine, if_exists="replace", index_label="family_id")

# === 7. dim_time ===
annees_cons = df_cons.columns.difference(["Famille de produits", "famille_nette"]).astype(int)

annees_sante = df_health["year_id"].unique()
years = pd.DataFrame({"year_id": sorted(set(annees_cons) | set(annees_sante))})
years.to_sql("dim_time", engine, if_exists="replace", index=False)

# === 8. dim_food ===
df_fam_db = pd.read_sql_table("dim_family", engine)
if "Famille_regroupee" not in df_food.columns:
    raise KeyError("❌ 'Famille_regroupee' est manquante dans DF_Aliment_NUTRIPROFIL.csv")

df_food = df_food.rename(columns={"Famille_regroupee": "family_name", "Aliment": "name"})
df_food = df_food.merge(df_fam_db, on="family_name", how="left")
df_food[["name", "Fibres", "Gras", "Glucides", "IG", "CG", "Proteines", "family_id"]].to_sql(
    "dim_food", engine, if_exists="replace", index_label="food_id"
)

# === 9. fact_consumption ===
df_melt = df_cons.melt(id_vars=["Famille de produits", "famille_nette"], var_name="year_id", value_name="value")
df_melt["year_id"] = df_melt["year_id"].astype(int)
df_melt = df_melt.merge(df_fam_db, left_on="famille_nette", right_on="family_name", how="left")
df_melt[["year_id", "family_id", "value"]].to_sql("fact_consumption", engine, if_exists="replace", index=False)

# === 10. fact_health ===
df_health.to_sql("fact_health", engine, if_exists="replace", index=False)

# === 11. food_health_link (exemples) ===
food_health_data = pd.DataFrame([
    {
        'food_id': 1,
        'indicateur': 'diabete_prev',
        'effet': '+9% risque de diabète',
        'seuil_description': 'par +10g/j de sucres ajoutés',
        'direction': 'augmentation',
        'source': 'GBD 2019'
    },
    {
        'food_id': 1,
        'indicateur': 'mcv_mort_rate',
        'effet': '+12% mortalité cardiovasculaire',
        'seuil_description': 'par 100ml/j de soda',
        'direction': 'augmentation',
        'source': 'Lancet Study'
    }
])
food_health_data.to_sql("food_health_link", engine, if_exists="replace", index=False)

print("✅ Base de données 'nutriprofil_final.db' générée avec succès.")


✅ Base de données 'nutriprofil_final.db' générée avec succès.


In [6]:
df_food['family_name'].unique()  # Afficher les familles uniques pour vérification

array(['Boissons', 'Produits animaux', 'Plats préparés',
       'Produits laitiers', 'Aides culinaires', 'Produits sucrés',
       'Fruits et légumes', 'Céréales', 'Fruits et lÃ©gumes', 'Autres'],
      dtype=object)

In [7]:
df_cons['famille'].unique()  # Afficher les familles uniques pour vérification

array(['Céréales et produits à base de céréales (riz, pâtes, etc)',
       'Viandes', 'Poissons et autres fruits de mer ',
       'Lait, produits laitiers divers et œufs ', 'Huiles et graisses ',
       'Fruits et fruits à coque ',
       'Légumes, tubercules, bananes plantains, bananes de cuisine et légumes secs ',
       'Sucre, confiserie et desserts ',
       'Plats préparés et autres produits alimentaires ',
       'Jus de fruits et de légumes ', 'Café et succédanés de café ',
       'Thé, maté et autres produits d’origine végétale pour tisanes ',
       'Eau ', 'Sodas', 'Autres boissons non alcoolisées ',
       'Spiritueux et liqueurs ', 'Vin', 'Bière ',
       'Autres boissons alcoolisées '], dtype=object)

In [10]:
print(df_cons.columns)


Index(['Famille de produits', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
       '2024', '2025', '2026', '2027'],
      dtype='object')
